In [1]:
import numpy as np
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 666)

from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
knn_clf.score(X_test, y_test)

0.9888888888888889

### Grid Search

In [9]:
#利用到了相关性，例如如果weights = uniform，就没有p这个参数
param_grid = [
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1,11)]
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1,11)],
        'p':[i for i in range(1,6)]
    }
]



In [10]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(knn_clf, param_grid)

In [11]:
%%time
grid_search.fit(X_train, y_train)

/Users/bessie/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 1min 37s, sys: 154 ms, total: 1min 37s
Wall time: 1min 37s


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
grid_search.best_estimator_ #返回的是KNeighborsClassifier

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=3,
           weights='distance')

In [13]:
grid_search.best_score_ 
#1.CV
#虽然跟上一章的best_score低，可是是因为上一章：我们用的是sum(xx==yy)/len(yy)
#这里是用的是CV：cross validation, 这是一个更准确的方法
#2. 代码原则：
#不是用户直接传入的参数，而是根据用户传入的参数计算过后得到的参数。每个单词后面跟一个下划线_

0.9853862212943633

In [14]:
grid_search.best_params_

{'n_neighbors': 3, 'p': 3, 'weights': 'distance'}

In [15]:
best_knn_clf = grid_search.best_estimator_ 

In [16]:
y_predict = best_knn_clf.predict(X_test)

In [17]:
best_knn_clf.score(X_test, y_test)

0.9833333333333333

### 性能上的提速: 多核

In [20]:
%%time
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs=-1, verbose = 2)
grid_search.fit(X_train, y_train)
#n_jobs: 几核,并行处理, n_jobs = -1 全部核都用于网格搜索
#verbose: 有时候搜索需要进行几个小时，可以输出搜索过程。当verbose数值越大，输出越详细。经验值为2
#60 candidates: uniform 一共有10种情况，distance中有10*5=50种情况

Fitting 3 folds for each of 60 candidates, totalling 180 fits


/Users/bessie/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   12.0s


CPU times: user 475 ms, sys: 235 ms, total: 710 ms
Wall time: 16.6 s


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   16.5s finished
